In [31]:
!pip install --upgrade lxml cssselect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
secrets_file = '/content/drive/MyDrive/Colab Notebooks/secrets.ini'

In [74]:
import os
import zipfile
import configparser
from pprint import pprint
import re

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

import pandas as pd
import lxml.html
from lxml import etree

In [14]:
conf = configparser.ConfigParser()
conf.read(secrets_file)
list(conf['secrets'].keys())

['openai_key']

In [21]:

def extract_zip(zip_path, dest):
    os.makedirs(dest, exist_ok=True)
    
    # Extract the contents of the zip file into the new folder
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(dest)

def extract_from_google_drive(file_name, dest):
    # Authenticate and create the PyDrive client
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    
    # Find the file in Google Drive
    file_list = drive.ListFile({'q': f"title='{file_name}'"}).GetList()
    if len(file_list) == 0:
        print(f"No file named {file_name} found in Google Drive")
        return
    
    # Download the file to a local temp directory
    file_id = file_list[0]['id']
    downloaded = drive.CreateFile({'id': file_id})
    downloaded.GetContentFile('temp.zip')
    
    # Check if the downloaded file is a zip file and extract it if it is
    if zipfile.is_zipfile('temp.zip'):
        extract_zip('temp.zip', dest)
    else:
        print(f"{file_name} is not a zip file")
        
    # Delete the temp file
    os.remove('temp.zip')


In [3]:
takeout = '<your takeout zip file>'
where = '<where to extract the files>'

In [5]:
extract_from_google_drive(takeout, where)

In [18]:
data_file = f'{where}Takeout/My Activity/Search/MyActivity.html'

In [ ]:
def extract_sections(html):
    parsed = lxml.html.fromstring(html)
    outer_cells = parsed.cssselect(".outer-cell.mdl-cell.mdl-cell--12-col.mdl-shadow--2dp")
    return outer_cells

In [ ]:
with open(data_file, 'r') as f:
    html = f.read()
    sections = extract_sections(html)
    print(f"Found {len(sections)} sections")

In [94]:
data = []
bad_entries = []
timestamp_pattern = r"(\b(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\b \d{1,2}, \d{4}, \d{1,2}:\d{2}:\d{2}(?:&#8239;)?(?:AM|PM) (?:EDT|EST|CDT|CST|MDT|MST|PDT|PST))"
for ele in sections:
    section_info = {
        'heading': ele.find_class('mdl-typography--title')[0].text_content(),
    }
    
    content = ele.find_class('content-cell mdl-cell mdl-cell--6-col mdl-typography--body-1')[0]
    content_text  = content.text_content()
    section_info['action'] = content_text.partition('\xa0')[0]
    section_info['timestamp'] = re.search(timestamp_pattern, etree.tostring(content).decode('utf8')).group(1).replace('&#8239;', '')

    try:
        a_tag = content.cssselect('a')[0]
        section_info['url'] = a_tag.get('href')
        caption = ele.cssselect('div.content-cell.mdl-cell.mdl-cell--12-col.mdl-typography--caption > a')
        if len(caption) > 1:
            section_info['location'] = caption[0].get('href')
        if section_info['action'].startswith('Searched for'):
            section_info['search_query'] = a_tag.text_content()
        else:
            section_info['site_title'] = a_tag.text_content()

    except IndexError:
        bad_entries.append(ele)

    data.append(section_info)


In [121]:
subset = '\n'.join([f"search_query: {d['search_query']}\ntimestamp: {d['timestamp']}\n" for d in data if 'search_query' in d][::100])
len(subset)
# print(subset)

11155